##### Bibliotecas


In [5]:
import os
import numpy as np
import pandas as pd
import openpyxl #biblioteca para ler o arquivo .xlsx
import plotly.express as px
import plotly.io as pio
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns
#from plotly.subplots import make_subplots

In [6]:
pio.renderers.default = 'notebook'  # Configura o renderizador para notebook

##### Globais

In [4]:
diretorio        = 'datasets'
arquivo_extensao = '.csv'
lista_arquivos   = []

___
##### Conjunto de Dados

Série Histórica de Preços de Combustíveis e de GLP   
https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/serie-historica-de-precos-de-combustiveis

A análise será realizada no **Conjunto de Dados semanais agrupados por semestre (2013 a 2023)**, considerando somente os **Combustíveis automotivos**, totalizando 21 arquivos csv.
<br><br>
___

##### Análise da Estrutura dos Dados (antes da limpeza)



In [ ]:
df_dados          = pd.DataFrame()    
df_estrutra_dados = pd.DataFrame()

# Lista os arquivos do diretório
for arquivo in os.listdir(diretorio):
    if arquivo.endswith(arquivo_extensao):
       lista_arquivos.append(arquivo)

# Carrega o DataFrame 
for dataset in lista_arquivos:
    caminho_arquivo   = os.path.join(diretorio, dataset)
    df_dados          = pd.read_csv(caminho_arquivo, sep=';', low_memory=False)
    df_estrutra_dados = pd.concat([df_estrutra_dados, df_dados], ignore_index=True)

print("Dimensões da base")
print("-----------------")
print("")
print(f"Linhas: {len(df_estrutra_dados)}")
print(f"Colunas: {len(df_estrutra_dados.iloc[0])}")
    
print("")
print("") 
print("Estrutura")
print("---------")
print("")
print(df_estrutra_dados.info())

print("")
print("") 
print("Qtde de campos nulos")
print("--------------------")
print("")
print(df_estrutra_dados.isnull().sum())

# Visualização do DataFrame (Head e Tail)
print(display(df_estrutra_dados)) 

##### Análise da Estrutura dos Dados (após a limpeza)

Otimização:
* Colunas não consideradas na análise serão excluídas do dataframe
* A fim de um melhor aproveitamento de espaço, algumas colunas terão o seu tipo alterado

In [ ]:
df_dados        = pd.DataFrame()
df_dados_limpos = pd.DataFrame()

for dataset in lista_arquivos:
    caminho_arquivo = os.path.join(diretorio, dataset)
    df_dados        = pd.read_csv(caminho_arquivo,
                                  sep=';', thousands = '.', decimal = ',', 
                                  dtype = {'Regiao - Sigla': 'category',
                                           'Estado - Sigla': 'category',
                                           'Valor de Compra': np.float64})
    
    df_dados_limpos = pd.concat([df_dados_limpos, df_dados], ignore_index=True)
    
df_dados_limpos.drop(columns=['Nome da Rua',
                              'Numero Rua',
                              'Complemento',
                              'Bairro',
                              'Cep',
                              'Valor de Compra'], inplace=True)    
df_dados_limpos.info()